In [1]:
from keras.models import Model
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, ZeroPadding1D, Conv1D, Dense, TimeDistributed, concatenate
from keras_contrib.layers import CRF
import keras.backend as K
from keras.callbacks import ModelCheckpoint,Callback
from keras.preprocessing.sequence import pad_sequences
import numpy as np


/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        

class NERModel:
    def __init__(self, maxlen, word_dict_size, word_vec_size, class_label_count):
        self.maxlen = maxlen
        self.word_dict_size = word_dict_size
        self.word_vec_size = word_vec_size
        self.class_label_count = class_label_count
        self.model = self._build_model()
        
    def _build_model(self):
        input_layer = Input(shape=(self.maxlen,), dtype='int32', name='input_layer')
        embedding_layer = Embedding(self.word_dict_size, self.word_vec_size, name='embedding_layer')(input_layer)
        bilstm = Bidirectional(LSTM(32, return_sequences=True))(embedding_layer)
        bilstm_d = Dropout(0.1)(bilstm)
        half_window_size = 2
        paddinglayer = ZeroPadding1D(padding=half_window_size)(embedding_layer)

        conv = Conv1D(nb_filter=50, filter_length=(2 * half_window_size + 1), border_mode='valid')(paddinglayer)
        conv_d = Dropout(0.1)(conv)
        dense_conv = TimeDistributed(Dense(50))(conv_d)
        rnn_cnn_merge = concatenate([bilstm_d, dense_conv], axis=2)
        dense = TimeDistributed(Dense(self.class_label_count))(rnn_cnn_merge)
        crf = CRF(self.class_label_count, sparse_target=False)
        crf_output = crf(dense)
        model = Model(input=[input_layer], output=[crf_output])
        model.compile(loss=crf.loss_function, optimizer='adam', metrics=[crf.accuracy])
        model.summary()
    
        return model
    
    def train(self, data, label):
        checkpointer = ModelCheckpoint(filepath="../model/bilstm_1102_k205_tf130.w", verbose=0, save_best_only=True, save_weights_only=True) #save_weights_only=True
        history = LossHistory()
        data = pad_sequences(data, self.maxlen)
        label = pad_sequences(label, self.maxlen,value=-1)
        label = np.expand_dims(label,2)
        print('---->',data.shape)
        print('---->',label.shape)
        self.model.fit(data, label,
                       batch_size=32, epochs=500,#validation_data = ([x_test, seq_lens_test], y_test),
                       callbacks=[checkpointer, history],
                       verbose=1,
                       validation_split=0.1,
                      )
        
    def predict(self, data):
        self.model.load_weights("../model/bilstm_1102_k205_tf130.w")
        result = mdoel.predict(data)
        return result

In [3]:
# maxlen = 10
# dict_size = 300
# word_vec_size = 300
# class_label_count = 4

import json
import numpy as np
class GetData:
    def __init__(self):
        self.data_save_path_root = '../data/'
        self.chunk_tags = ['B_AT', 'I_AT', 'O', 'B_OT', 'I_OT']
        self.chunk2id, self.id2chunk = self._reshape_data('all')
    def _reshape_data(self, mode):
        """
            mode ->  'all' : 标注在一起
                     ->  'at' : AspectTerm only
                     ->  'ot' : OpinionTerms only
        """
        if mode == 'all':
            chunk_tags = self.chunk_tags
        elif mode == 'at':
            chunk_tags = self.chunk_tags[:3]
        elif mode == 'ot':
            chunk_tags = self.chunk_tags[2:]
            
        chunk2id = {item: _id for _id, item in enumerate(chunk_tags)}
        id2chunk = {v: k for k, v in chunk2id.items()}
        
        return chunk2id, id2chunk
    
    def get_train_data(self, granularity):
        with open(self.data_save_path_root + granularity + '_level.json', 'r', encoding='utf8') as fr:
            vocab_dict = json.load(fr)
        with open(self.data_save_path_root + 'ner_' + granularity + '_data.json', 'r', encoding='utf8') as fr:
            ner_data = json.load(fr)
        with open(self.data_save_path_root + 'ner_' + granularity + '_label.json', 'r', encoding='utf8') as fr:
            ner_label = json.load(fr)
            
            
        vocab2id = vocab_dict.get(granularity + '2id')
        id2vocab = vocab_dict.get('id2' + granularity)
        
        for i in range(len(ner_data)):
            ner_data[i] = np.asarray([vocab2id.get(item, 1) for item in ner_data[i]])
            
        ner_data = np.asarray(ner_data)
        
        for i in range(len(ner_label)):
            ner_label[i] = np.asarray([self.chunk2id.get(item, 1) for item in ner_label[i]])
            
        ner_data = np.asarray(ner_data)
        ner_label = np.asarray(ner_label)
        
        
        return self.chunk2id, self.id2chunk, vocab2id, id2vocab, ner_data, ner_label

In [4]:
get_data_obj = GetData()
chunk2id, id2chunk, vocab2id, id2vocab, ner_data, ner_label = get_data_obj.get_train_data('char')


In [5]:
ner = NERModel(maxlen = 30, word_dict_size = len(vocab2id), word_vec_size = 128, class_label_count = len(chunk2id)+1)
ner.train(ner_data, ner_label )

W0820 18:35:10.688950 4384560576 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 18:35:10.702543 4384560576 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 18:35:10.713519 4384560576 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 18:35:11.260151 4384560576 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 30, 128)      165120      input_layer[0][0]                
__________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D (None, 34, 128)      0           embedding_layer[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 50)       32050       zero_padding1d_1[0][0]           
__________________________________________________________________________________________________
bidirectio

W0820 18:35:13.552367 4384560576 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 2906 samples, validate on 323 samples
Epoch 1/500


InvalidArgumentError: Matrix size-incompatible: In[0]: [928,1], In[1]: [6,6]
	 [[{{node loss/crf_1_loss/MatMul_1}}]]